In [49]:
import pickle
import pandas as pd
import numpy as np
import datetime  
import pickle
from math import floor
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor, Lasso, Ridge
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from sqlalchemy import text

Inserting data base information and connecting to it

In [36]:

URI = "database-2.ckmnj1f5m6qh.eu-west-1.rds.amazonaws.com"
PORT = "3306"
DB = "backupdata"
USER = "admin"
PASSWORD = "DublinBikes3"

engine = create_engine("mysql://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

Want to get features to compare in train_test split, I will use datetime to try predict availability first

In [37]:
# Gets features in arrays of [station number, days, hours]
def get_features():
    # Get datetime and station number for data
    features_raw = engine.execute("SELECT datetime, number FROM availability2;")

    # Create list to store station numbers, days and hours in.
    numbers_days_hours = []
    for row in features_raw:
        weather_raw = engine.execute("SELECT wind_speed FROM weather2 WHERE datetime BETWEEN %s AND %S", (row[0]-1, row[0]+2))
        # Will store each trio in
        number_day_hour_wind = []
        
        # Split epoch time into day and hour
        epoch_time = row[0]
        day = (floor(epoch_time / 86400) + 4) % 7
        number_day_hour.append(row[1])
        number_day_hour.append(day)
        number_day_hour.append(datetime.datetime.fromtimestamp(row[0]).hour)
        numbers_days_hours.append(number_day_hour)
    return numbers_days_hours

def get_availability():
    availability_raw = engine.execute("SELECT available_bikes FROM availability2;")
    # This returns the number of available bikes
    availability = []
    for row in availability_raw:
        availability.append(row[0])
    return availability
                                     

In [38]:

X = get_features()
y = get_availability()

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train

2023-04-11 08:49:10,808 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-11 08:49:10,809 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-11 08:49:10,821 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-11 08:49:10,822 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-11 08:49:10,828 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-11 08:49:10,829 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-11 08:49:10,841 INFO sqlalchemy.engine.Engine SELECT datetime, number FROM availability2;
2023-04-11 08:49:10,841 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-11 08:49:15,832 INFO sqlalchemy.engine.Engine SELECT available_bikes FROM availability2;
2023-04-11 08:49:15,833 INFO sqlalchemy.engine.Engine [raw sql] ()


[[1, 2, 12],
 [32, 3, 12],
 [11, 5, 23],
 [56, 5, 6],
 [26, 3, 10],
 [30, 0, 12],
 [29, 6, 17],
 [48, 6, 19],
 [59, 5, 0],
 [72, 1, 5],
 [22, 2, 22],
 [5, 3, 18],
 [108, 6, 4],
 [100, 4, 23],
 [99, 1, 2],
 [66, 5, 19],
 [52, 0, 6],
 [94, 3, 17],
 [106, 1, 18],
 [59, 5, 21],
 [27, 6, 16],
 [97, 2, 9],
 [66, 6, 12],
 [83, 4, 5],
 [29, 2, 8],
 [3, 3, 16],
 [6, 3, 4],
 [88, 2, 18],
 [102, 6, 3],
 [3, 6, 16],
 [52, 5, 15],
 [11, 0, 0],
 [92, 1, 6],
 [47, 5, 11],
 [16, 4, 12],
 [13, 2, 4],
 [113, 2, 19],
 [45, 4, 23],
 [57, 3, 5],
 [83, 4, 21],
 [84, 1, 10],
 [47, 0, 5],
 [25, 5, 2],
 [21, 0, 14],
 [44, 0, 3],
 [99, 2, 3],
 [77, 3, 13],
 [84, 4, 12],
 [108, 6, 3],
 [85, 3, 14],
 [111, 1, 10],
 [99, 2, 10],
 [5, 3, 21],
 [56, 1, 8],
 [109, 1, 15],
 [109, 6, 4],
 [15, 4, 2],
 [3, 3, 17],
 [85, 3, 15],
 [65, 4, 0],
 [77, 2, 1],
 [67, 0, 0],
 [28, 5, 6],
 [35, 6, 5],
 [88, 0, 10],
 [73, 0, 10],
 [48, 1, 6],
 [11, 2, 16],
 [116, 6, 10],
 [102, 2, 11],
 [78, 1, 3],
 [20, 5, 22],
 [63, 0, 17],
 [55

In [39]:
def test_model(model):
    # Fit model
    model.fit(X_train, y_train)
    
    # make a prediction
    ynew = model.predict(X_test)
    
    avg_difference = 0
    number_right = 0
    for i in range(len(ynew)):
        avg_difference += abs(y_test[i] - ynew[i])
        # Range we'll accept as right
        close_Enough = range(y_test[i] - 1, y_test[i] + 1)

        if int(ynew[i]) in close_Enough:
            number_right += 1
    
    perc_right = number_right / len(ynew)
    avg_difference /= len(ynew)
    avg_difference
    print("Average difference is", avg_difference, "Percent within 1 +- is :", perc_right)

In [40]:
test_model(SGDRegressor())

Average difference is 32.625562316390855 Percent within 1 +- is : 0.014002405279349688


In [41]:
test_model(LinearRegression())

Average difference is 7.694346408425275 Percent within 1 +- is : 0.07160986777777399


In [43]:
test_model(LogisticRegression())

Average difference is 11.54558617611687 Percent within 1 +- is : 0.13874646107092847


/Users/josephtummon/mambaforge/envs/comp30380/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [47]:
test_model(Lasso())

Average difference is 7.694982422732699 Percent within 1 +- is : 0.07057416675467854


In [50]:
test_model(Ridge())

Average difference is 7.6943464084287125 Percent within 1 +- is : 0.07160986777777399


In [42]:
# X_train and y_train are the training data
model = SGDRegressor()
model.fit(X_train, y_train)
# Serialize model object into a file called model.pkl on disk using pickle
with open('model.pkl', 'wb') as handle:
    pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)
# pickle.HIGHEST_PROTOCOL using the highest available protocol
# (we used wb to open file as binary and use a higher pickling protocol)
